#1. Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#2. Installing Packages

In [ ]:
!pip install SpeechRecognition
!pip install google-cloud-speech pandas
!pip install pydub
!apt install ffmpeg
!pip install FER

#3. Dataset Exploration



*   Importing Libraries
*   List item



In [ ]:
import os
import speech_recognition as sr
import spacy
import pandas as pd
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
from google.cloud import speech
import pandas as pd
import os
import glob
from pydub import AudioSegment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from fer import FER

##Labled Transcriptions



*   **updated_transcriptions_with_senti.csv**
*   Senti coloum 0 values unique



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/work2/updated_transcriptions_with_senti.csv')
df.head(111)

In [ ]:
df[df['Senti'] == 0].head(111)


In [ ]:
count = df[df['Senti'] == 0].shape[0]
print(count)

##Saved audio analysis in **transcription_results_final.csv**

In [ ]:
import os
import pandas as pd
import speech_recognition as sr
from pydub import AudioSegment

def get_audio_length(file_path):
    """Return the length of the audio file in seconds."""
    audio = AudioSegment.from_file(file_path)
    return len(audio) / 1000.0  # Convert milliseconds to seconds

def transcribe_audio_speech_recognition(audio_file_path):
    """Transcribe the given local audio file using the speech_recognition library."""
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file_path) as source:
        audio_data = recognizer.record(source)
        try:
            return recognizer.recognize_google(audio_data, language="ur-PK")
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return None

def read_words_from_file(file_path):
    """Read words from a file and return them as a list."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return [line.strip().lower() for line in file.readlines()]
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []

def analyze_sentiment(words_in_transcript, negative_words, positive_words):
    negative_count, positive_count = 0, 0
    negative_positions, positive_positions = [], []
    for index, word in enumerate(words_in_transcript):
        position = (index + 1) / len(words_in_transcript) * length_in_seconds
        if word.lower() in negative_words:
            negative_count += 1
            negative_positions.append(round(position))
        elif word.lower() in positive_words:
            positive_count += 1
            positive_positions.append(round(position))
    return negative_count, positive_count, negative_positions, positive_positions

# Path to the folder containing audio files
audio_folder_path = '/content/drive/MyDrive/work2/wav'
negative_words = read_words_from_file('/content/drive/MyDrive/work2/dictionary/urdu_negative_words.txt')
positive_words = read_words_from_file('/content/drive/MyDrive/work2/dictionary/urdu_positive_words.txt')

results = []
for audio_file in os.listdir(audio_folder_path):
    if audio_file.endswith('.wav'):
        audio_file_path = os.path.join(audio_folder_path, audio_file)
        length_in_seconds = get_audio_length(audio_file_path)
        transcript = transcribe_audio_speech_recognition(audio_file_path)

        if transcript:
            words_in_transcript = transcript.split()
            negative_count, positive_count, negative_positions, positive_positions = analyze_sentiment(words_in_transcript, negative_words, positive_words)

            results.append({
                'Length': length_in_seconds,
                'audio_name': audio_file,
                'Transcript': transcript,
                'negative_count': negative_count,
                'positive_count': positive_count,
                'negative_positions': negative_positions,
                'positive_positions': positive_positions
            })

# Save results to a CSV file
csv_file_path = '/content/drive/MyDrive/work2/Final/transcription_results_final.csv'
df = pd.DataFrame(results)
df.to_csv(csv_file_path, index=False)
print(f"Results saved to {csv_file_path}")


##Show transcription_results_final.csv

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/work2/Final/transcription_results_final.csv')
df.head(12)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/work2/Final/transcription_results_final.csv')
df.info()

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/work2/Final/transcription_results_final.csv')
df.head()

## Extraction of all FRAMES

In [ ]:
import cv2
import pandas as pd
import os

# Function to extract frames
def extract_frames(video_file_path, output_folder):
    video_name = os.path.basename(video_file_path).split('.')[0]
    cap = cv2.VideoCapture(video_file_path)
    frame_info = []
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = int(total_frames / frame_rate)

    for current_time in range(duration + 1):
        cap.set(cv2.CAP_PROP_POS_MSEC, current_time * 1000)
        ret, frame = cap.read()
        if not ret:
            break
        frame_filename = os.path.join(output_folder, f'{video_name}_frame_{current_time}.jpg')
        cv2.imwrite(frame_filename, frame)
        frame_info.append({'Video Name': video_name, 'Frame Path': frame_filename, 'Time in Seconds': current_time})

    cap.release()
    return frame_info

# Directory containing videos
video_directory = '/content/drive/MyDrive/work2/videos'

output_base_folder_Training = '/content/drive/MyDrive/work2/Training_Data/train_frames/All_Frames'
output_base_folder_Testing = '/content/drive/MyDrive/work2/Testing_Data/test_frames/All_Frames'

all_frame_info = []  # Store info for all frames from all videos

# Process each video file
video_files = sorted(os.listdir(video_directory))
for i, video_file in enumerate(video_files):
    video_file_path = os.path.join(video_directory, video_file)
    video_name = os.path.splitext(video_file)[0]

    # Skip files that are not named with numbers (e.g., 1.mp4, 2.mp4, etc.)
    if not video_name.isdigit():
        continue

    video_num = int(video_name)

    # Determine output folder based on video file number
    if 1 <= video_num <= 77:
        output_folder = os.path.join(output_base_folder_Training, video_name)
    elif 78 <= video_num <= 111:
        output_folder = os.path.join(output_base_folder_Testing, video_name)
    else:
        continue

    # Create output folder for this video
    os.makedirs(output_folder, exist_ok=True)

    # Extract frames
    frame_info = extract_frames(video_file_path, output_folder)
    all_frame_info.extend(frame_info)

# Save frame information to a new CSV file
frames_csv_path = '/content/drive/MyDrive/work2/Final/Extraction_Frames_results_final.csv'
frame_info_df = pd.DataFrame(all_frame_info)
frame_info_df.to_csv(frames_csv_path, index=False)

print("Frames extracted and saved.")
print(f"Frame information saved to {frames_csv_path}")


## Extraction of Key Frames

In [ ]:
import cv2
import pandas as pd
import os

# Function to extract frames at specified times
def extract_frames(video_file_path, times_in_seconds, output_folder):
    video_name = os.path.basename(video_file_path).split('.')[0]
    cap = cv2.VideoCapture(video_file_path)
    frame_info = []
    extracted_times = set()
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        current_time = int(cap.get(cv2.CAP_PROP_POS_MSEC) // 1000)
        if current_time in times_in_seconds and current_time not in extracted_times:
            frame_filename = os.path.join(output_folder, f'{video_name}_frame_{current_time}.jpg')
            cv2.imwrite(frame_filename, frame)
            frame_info.append({'Video Name': video_name, 'Frame Path': frame_filename, 'Time in Seconds': current_time})
            extracted_times.add(current_time)
    cap.release()
    return frame_info

# Directory containing videos
video_directory = '/content/drive/MyDrive/work2/videos'

output_base_folder_Training = '/content/drive/MyDrive/work2/Training_Data/train_frames/Key_Frames'
output_base_folder_Testing = '/content/drive/MyDrive/work2/Testing_Data/test_frames/Key_Frames'

# Load positions from the CSV file
csv_file_path = '/content/drive/MyDrive/work2/Final/transcription_results_final.csv'
df = pd.read_csv(csv_file_path)

# Convert positions to lists of integers
df['negative_positions'] = df['negative_positions'].apply(lambda x: eval(x))
df['positive_positions'] = df['positive_positions'].apply(lambda x: eval(x))

all_frame_info = []  # Store info for all frames from all videos

# Process each video file
video_files = sorted(os.listdir(video_directory))
for i, video_file in enumerate(video_files):
    video_file_path = os.path.join(video_directory, video_file)
    video_name = os.path.splitext(video_file)[0]

    # Skip files that are not named with numbers (e.g., 1.mp4, 2.mp4, etc.)
    if not video_name.isdigit():
        continue

    video_num = int(video_name)

    # Determine output folder based on video file number
    if 1 <= video_num <= 77:
        output_folder = os.path.join(output_base_folder_Training, video_name)
    elif 78 <= video_num <= 111:
        output_folder = os.path.join(output_base_folder_Testing, video_name)
    else:
        continue

    # Create output folder for this video
    os.makedirs(output_folder, exist_ok=True)

    # Filter the dataframe for the current video
    video_df = df[df['audio_name'].str.contains(video_name)]

    # Concatenate, flatten, and remove duplicates from positions
    all_positions = video_df['negative_positions'].tolist() + video_df['positive_positions'].tolist()
    times_in_seconds = list(set([int(time) for sublist in all_positions for time in sublist]))

    # Extract frames
    frame_info = extract_frames(video_file_path, times_in_seconds, output_folder)
    all_frame_info.extend(frame_info)

# Save frame information to a new CSV file
frames_csv_path = '/content/drive/MyDrive/work2/Final/Extraction_Key_Frames_results_final.csv'
frame_info_df = pd.DataFrame(all_frame_info)
frame_info_df.to_csv(frames_csv_path, index=False)

print("Frames extracted and saved.")
print(f"Frame information saved to {frames_csv_path}")


#FER on All Frames

In [ ]:
import pandas as pd
import cv2
from fer import FER

# Function to analyze the most prominent face in a frame and categorize emotions
def analyze_most_prominent_face(frame_path):
    try:
        detector = FER(mtcnn=True)
        image = cv2.imread(frame_path)
        face_detections = detector.detect_emotions(image)

        if face_detections:
            largest_face_area = 0
            positive_sentiment = 0
            negative_sentiment = 0
            neutral_sentiment = 0

            # Find the most prominent face in the frame
            for face in face_detections:
                x, y, w, h = face["box"]
                face_area = w * h
                if face_area > largest_face_area:
                    largest_face_area = face_area
                    emotions = face["emotions"]

                    # Categorize emotions into positive and negative sentiments
                    if any(emotions.get(emotion, 0) > 0.5 for emotion in ['happy', 'surprise']):
                        positive_sentiment = 1
                    if any(emotions.get(emotion, 0) > 0.5 for emotion in ['angry', 'fear', 'disgust']):
                        negative_sentiment = 2
                    if any(emotions.get(emotion, 0) > 0.5 for emotion in ['neutral', 'sad']):
                        neutral_sentiment = 0

            return positive_sentiment, negative_sentiment,neutral_sentiment
        else:
            return 9, 9 ,9  # Neutral for no face
    except Exception as e:
        print(f"Error in analyzing frame {frame_path}: {e}")
        return 'Error', 'Error'

# Load transcription and frame details DataFrames
transcription_df = pd.read_csv('/content/drive/MyDrive/work2/Final/transcription_results_final.csv')
frame_details_df = pd.read_csv('/content/drive/MyDrive/work2/Final/extracted_frames_info.csv')

# Convert positions in transcription data to lists of integers
transcription_df['negative_positions'] = transcription_df['negative_positions'].apply(lambda x: eval(x))
transcription_df['positive_positions'] = transcription_df['positive_positions'].apply(lambda x: eval(x))

# Initialize sentiment columns in frame details DataFrame
frame_details_df['Positive Sentiment'] = None
frame_details_df['Negative Sentiment'] = None

# Analyze frames for sentiment and assign to corresponding positions
for index, row in frame_details_df.iterrows():
    frame_time = row['Time in Seconds']
    frame_path = row['Frame Path']
    positive_sentiment, negative_sentiment,neutral_sentiment = analyze_most_prominent_face(frame_path)

    if any(frame_time in positions for positions in transcription_df['positive_positions']):
        frame_details_df.at[index, 'Positive Sentiment'] = positive_sentiment
    if any(frame_time in positions for positions in transcription_df['negative_positions']):
        frame_details_df.at[index, 'Negative Sentiment'] = negative_sentiment

# Save updated frame details with FER analysis results
frame_details_with_sentiment_path = '/content/drive/MyDrive/work2/Final/frame_details_with_sentiment.csv'
frame_details_df.to_csv(frame_details_with_sentiment_path, index=False)
print(f"Updated frame details with FER analysis saved to {frame_details_with_sentiment_path}")

def compile_sentiments(row, frame_details):
    positive_sentiments = []
    negative_sentiments = []

    # Use set to ensure unique positions are considered
    unique_negative_positions = set(row['negative_positions'])
    unique_positive_positions = set(row['positive_positions'])

    for pos in unique_negative_positions:
        sentiment = frame_details.loc[frame_details['Time in Seconds'] == pos, 'Negative Sentiment'].values
        if sentiment.size > 0:
            negative_sentiments.append(sentiment[0])  # Append the first sentiment found

    for pos in unique_positive_positions:
        sentiment = frame_details.loc[frame_details['Time in Seconds'] == pos, 'Positive Sentiment'].values
        if sentiment.size > 0:
            positive_sentiments.append(sentiment[0])  # Append the first sentiment found

    return pd.Series([negative_sentiments, positive_sentiments])

# Compile sentiments into transcription DataFrame
transcription_df[['Negative_Frame_Sentiments', 'Positive_Frame_Sentiments']] = transcription_df.apply(
    lambda row: compile_sentiments(row, frame_details_df), axis=1
)

# Save updated transcription results with compiled sentiments
transcription_results_with_compiled_sentiments_path = '/content/drive/MyDrive/work2/Final/transcription_results_with_compiled_sentiments.csv'
transcription_df.to_csv(transcription_results_with_compiled_sentiments_path, index=False)
print(f"Transcription results with compiled sentiments saved to {transcription_results_with_compiled_sentiments_path}")